Small notebook for checking schemes and QE XML output consistency with the test-suite cases.  
For running:  
* adapt commands in process file routine
* in `q-e/test-suite/ENVIRONMENT`:
  - uncomment line 13 and comment line 14 
  - make sure that ESPRESSO_BUILD points to a directory with executables
* install xmlschema package in your python environment. 

In [ ]:
from xmlschema import XMLSchema, XMLSchemaChildrenValidationError
from xml.etree import ElementTree as et
from pathlib import Path
from glob import glob
import subprocess
import pickle

In [17]:
schema=XMLSchema('/data/pdelugas/repositories/qeschemas/PW_CPV/test_schemas/qes_dev_250412.xsd', validation='skip')

In [18]:
def process_file(file, arg1, directory, schema, outdir, 
				 stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL):
	input_file = str(directory / file)
	print(input_file)
	commands = f"""
	module load intel/2022.2.1
	module load mkl 
	module load openmpi4
	cd /scratch/pdelugas/qe_gitlab/test-suite/
	source ENVIRONMENT
	export QE_USE_MPI=4
	/scratch/pdelugas/qe_gitlab/test-suite/run-pw.sh {arg1} {input_file} o e
	"""
	subprocess.run(['bash', '-c', commands], stdout=stdout, stderr=stderr)
	errors = schema.decode(outdir / 'pwscf.xml', validation='lax')[1:]
	if len(errors[0]) > 0:
		print(errors[0][0].msg)
		return (file, errors)
	else:
		print (f"{file} run is xml ok")
	return None



In [19]:

def input_args(subdir):
	jobconfig={'pw_berry':[('berry.in', '1'), ('berry-1.in', '1'), ('berry-2.in','1')],
						 'pw_noncolin':[('noncolin.in' ,'1'), ('noncolin-1.in' ,'1'), ('noncolin-2.in' ,'1'), ('noncolin-cg.in' ,'1'), ('noncolin-rmm.in','1'), 
											('noncolin-constrain_angle.in' ,'1'), ('noncolin-constrain_atomic.in' ,'1'), ('noncolin-constrain_total.in' ,'1'), 
											('noncolin-hyb.in' ,'1'), ('noncolin-pbe.in','1')  ],
						 'pw_metal': [ ('metal.in' ,'1'), ('metal-2.in' ,'1'), ('metal-fermi_dirac.in' ,'1'), ('metal-gaussian.in' ,'1'), ('metal-tetrahedra.in' ,'1'),
										 ('metal-tetrahedra-1.in' ,'1'), ('metal-tetrahedra-2.in','1') ],
						 'pw_electric': [('electric.in' ,'1'), ('electric-1.in' ,'1'), ('electric-2.in' ,'1')],
						 'pw_twochem': [('scf_twochem.in', '1'), ('nscf_twochem.in','2'), ('relax_twochem.in','1'), ('vc-relax_twochem.in','1') ],
						 'pw_spinorbit':[ ('spinorbit.in' ,'1'), ('spinorbit-1.in' ,'1'), ('spinorbit-3.in' ,'1'), ('spinorbit-pbe.in' ,'1'), 
											 ('spinorbit-paw.in','1') ],
						 'pw_uspp': [ ('uspp.in' ,'1'), ('uspp-2.in' ,'1'), ('uspp-cg.in' ,'1'), ('uspp-cg-gamma.in' ,'1'), ('uspp-hyb-g.in' ,'1'),
									  ('uspp-hyb-k.in','1'), ('uspp-mixing_TF.in' ,'1'), ('uspp-mixing_localTF.in' ,'1'), ('uspp-mixing_ndim.in' ,'1'), 
										('uspp-singlegrid.in' ,'1'), ('uspp1-coulomb.in' ,'1'), ('uspp1.in' ,'1'), ('uspp2.in' ,'1'), ('uspp-paro-gamma.in','1'), 
										('uspp-paro-k.in','1'), ('uspp-ppcg-gamma.in','1'), ('uspp-ppcg-k.in','1')  ],
						 'pw_electric' : [ ('electric.in' ,'1'), ('electric-1.in' ,'1'), ('electric-2.in' ,'1') ],
						 'pw_scf'      : [ ('scf.in' ,'1'), ('scf-1.in' ,'1'), ('scf-2.in' ,'1'), ('scf-allfrac.in' ,'1'), ('scf-cg.in' ,'1'), 
						                   ('scf-cg-gamma.in' ,'1'), ('scf-disk_io.in' ,'1'), ('scf-disk_io-1.in' ,'1'), ('scf-disk_io-2.in' ,'1'), ('scf-gamma.in' ,'1'),
										   ('scf-k0.in' ,'1'), ('scf-kauto.in' ,'1'), ('scf-kcrys.in' ,'1'), ('scf-mixing_TF.in' ,'1'), ('scf-mixing_beta.in' ,'1'), 
										   ('scf-mixing_localTF.in' ,'1'), ('scf-mixing_ndim.in' ,'1'), ('scf-ncpp.in' ,'1'), ('scf-nofrac.in' ,'1'), ('scf-occ.in' ,'1'), 
										   ('scf-paro-gamma.in','1'), ('scf-paro-k.in','1'), ('scf-rmm-gamma.in','1'), ('scf-rmm-paro-gamma.in','1'), ('scf-rmm-k.in','1'), 
										   ('scf-rmm-paro-k.in','1'), ('scf-ppcg-gamma.in','1'), ('scf-ppcg-k.in','1'), ('scf-gth.in','1')  ]
						}
	print (subdir.name)
	if subdir.name in jobconfig.keys():
		return jobconfig[subdir.name]
	else:
		return [(file,1) for file in subdir.glob('*.in') if not file.name.startswith('benchmark')]

In [ ]:
from pathlib import Path
directory = Path('/scratch/pdelugas/qe_gitlab/test-suite')
outdir = directory
results = [
	process_file(file, arg1, subdir, schema, outdir)
	for subdir in [_ for _ in directory.glob('pw_atom')] 
	for file, arg1 in  input_args(subdir) 
]

issues = [result for result in results if result is not None]

In [21]:
from datetime import datetime 
date = datetime.today()
y,d,h,m,s = date.year, date.day, date.hour, date.minute, date.second
with open(f'issues_{y}_{d}_{h}_{m}_{s}.pkl', 'wb') as f:
	pickle.dump([_[0] for _ in issues], f)

In [22]:

def filter_cases(cases, message):
    return [case for case in cases
            if True in [message in _
                        for _ in [__.msg for __ in case[1][0]]
                        ]
    ]
def extract_issues(cases):
    from itertools import chain
    return set([msg  for msg in [msg_ for msg_ in [__[1][0][0].msg for __ in cases]]])
               

In [ ]:
extract_issues(issues)